In [1]:
G = CartanType('A2')
G.dynkin_diagram()

O---O
1   2   
A2

In [2]:
L = CartanType('A1')
L.dynkin_diagram()

O
1   
A1

In [3]:
crossed_out_nodes = [1]
crossed_out_nodes.sort()
crossed_out_nodes

[1]

In [4]:
G.marked_nodes(crossed_out_nodes).dynkin_diagram()

X---O
1   2   
A2 with node 1 marked

Correction function which converts index of L to that of G.

In [5]:
def correct_index(index: int) -> int:
    for i in range(len(crossed_out_nodes)):
        if index + i < crossed_out_nodes[i]:
            return index + i
    return index + len(crossed_out_nodes)

In [6]:
[ correct_index(k) for k in L.index_set() ]

[2]

In [7]:
R_G = WeylCharacterRing(G)
R_L = WeylCharacterRing(L)

In [8]:
def root_difference_multiplicities(character_ring, highest_weight) -> dict:
    weight_muliplicities = character_ring(highest_weight).weight_multiplicities()
    A = matrix([ vector(sr) for sr in character_ring.simple_roots() ]).transpose()

    result ={}

    for k, v in weight_muliplicities.items():
        Y = vector(k) - vector(highest_weight)
        result[tuple(A.solve_right(Y))] = v
    
    return result

In [9]:
R = PolynomialRing(ZZ, 'x', G.root_system().ambient_space().dimension())
R

Multivariate Polynomial Ring in x0, x1, x2 over Integer Ring

In [10]:
x = R.gens()
x

(x0, x1, x2)

The below variable `weight` means a list or tuple of coefficients, so `weight` stands for an actual weight
`sum( weight[i] * fundamental_weight[i] for i)`

In [11]:
def parabolic_weight_muliplicities(weight) -> dict:
    fws_L = [fw for fw in R_L.fundamental_weights()] # conversin from 1-index to 0-index
    weight_for_L = [ weight[i - 1] for i in set(1..len(weight))-set(crossed_out_nodes)] 
    weight_for_L = sum( weight_for_L[i] * fws_L[i] for i in range(L.rank()))

    fws_G = [fw for fw in R_G.fundamental_weights()]; fws_G
    weight_for_G = sum( weight[i] * fws_G[i] for i in range(G.rank()))

    mul_set = root_difference_multiplicities(R_L, weight_for_L)

    result = {}
    for k, v in mul_set.items():
        w = weight_for_G + sum(k[i - 1] * R_G.simple_roots()[correct_index(i)] for i in (1..L.rank()))
        result[w] = v

    return result

In [12]:
parabolic_weight_muliplicities((1, 1))

{(2, 1, 0): 1, (2, 0, 1): 1}

!!A-type only!!

In [13]:
def class_from_weight(character_ring, weight):
    A = matrix([ vector(sr) for sr in character_ring.fundamental_weights() ]).transpose()
    v = A.solve_right(vector(weight) - vector([weight[-1]]*len(weight)))

    return sum(v[i] * x[i] for i in range(character_ring.rank()))

In [14]:
class_from_weight(R_G, (2, 0, 1))

2*x0 - x1

In [15]:
class_from_weight(R_G, (2, 1, 0))

x0 + x1

In [16]:
parabolic_weight_muliplicities((1, 1))

{(2, 1, 0): 1, (2, 0, 1): 1}

In [17]:
[ w for w, i in parabolic_weight_muliplicities((1, 1)).items() ]

[(2, 1, 0), (2, 0, 1)]

In [18]:
[ vector(w) for w, i in parabolic_weight_muliplicities((1, 1)).items() ]

[(2, 1, 0), (2, 0, 1)]

In [19]:
[ class_from_weight(R_G, vector(w)) for w, i in parabolic_weight_muliplicities((1, 1)).items() ]

[x0 + x1, 2*x0 - x1]

In [20]:
def chern_classes_of_GP(weight_muliplicities):
    return prod((1 + class_from_weight(R_G, vector(w)))^i for w, i in weight_muliplicities.items())

In [21]:
chern_classes_of_GP(parabolic_weight_muliplicities((1, 1)))

2*x0^2 + x0*x1 - x1^2 + 3*x0 + 1

In [22]:
chern_classes_of_GP(parabolic_weight_muliplicities((1, 1)))

2*x0^2 + x0*x1 - x1^2 + 3*x0 + 1

In [23]:
chern_classes_of_GP(parabolic_weight_muliplicities((1, 1)))

2*x0^2 + x0*x1 - x1^2 + 3*x0 + 1

In [24]:
Delta_G = G.root_system()

#ambient spaceを生成
L=Delta_G.ambient_space()

#positive rootをambient spaceの元に変換
P_G=[ L(pr) for pr in Delta_G.root_lattice().positive_roots() ];P_G

[(1, -1, 0), (0, 1, -1), (1, 0, -1)]

In [25]:
parabolic_weight_muliplicities((1, 1))

IndexError: list index out of range

In [26]:
L.rank()

3

In [488]:
chern_classes_of_GP(parabolic_weight_muliplicities((1, 1)))

IndexError: list index out of range

In [ ]:
# positive rootからcutoutされるsimple rootを引き, positiveでなければuncrossed nodeから生成されると判定
Pi_H=[ pr for pr in Delta_G.root_lattice().positive_roots() ]
for i in crossed_out_nodes:
    Pi_H=[ pr for pr in Pi_H if not (pr - (Delta_G.root_lattice().simple_roots())[i]).is_positive_root() ]
    # ambient spaceの元に変換
P_H=[ L(pr) for pr in Pi_H ]; P_H

[(0, 1, -1)]

In [ ]:
# the weights of the tangenst spaces at torus-fixed points
flag_tangent_weights = set(P_G)-set(P_H)
flag_tangent_weights = [sum( r[l]*x[l] for l in range(len(x))) for r in flag_tangent_weights]
flag_tangent_weights

[x0 - x2, x0 - x1]

In [ ]:
chern_classes_of_GP(parabolic_weight_muliplicities((1, 1)))

IndexError: list index out of range

In [ ]:
def numerical_integration_by_localization(f):
    random_x = [RealField(1000)(random()) for i in range(G.rank())]
    orbit_of_random_x = [(w.inverse()*vector(RealField(1000),random_x)).list() for w in WeylGroup(G)]
    dim_flag = len(R_G.positive_roots()) - len(R_L.positive_roots())
    top_of_f = homogeneous_part(f, dim_flag)
    denominator_in_localization = prod(flag_tangent_weights)
    return sum([top_of_f(x)/denominator_in_localization(x) for x in orbit_of_random_x]).round()

In [ ]:
chern_classes_of_GP(parabolic_weight_muliplicities((1, 1)))

IndexError: list index out of range